### trend visualization

In [30]:
# load data
library(ncdf4)
library(ggplot2)
library(lubridate)

database <- '/home/jianj0c/dataset/redsea/'
data_folder  <- 'Chlorophyll/Daily/'
dir_path <- paste(database,data_folder,"Daily_Aqua_MODIS_chloA.2013_2022.nc",sep="")
nc_obj <- nc_open(dir_path)

T_array <- ncvar_get(nc_obj,"chlorophyllA")
dim(T_array)

[1] 268
[1] 423
[1] 3465


In [37]:
# filter the outlier value based on xx method
pipeline_filter_for_outlier <- function(array) { # create a function with the name my_function
  print("Hello World!")
  return array
}


# take one 3d arrayday reduce the 3d data to 2d
reducer_trend<- function(array) { 
  print("Hello World!")
  return array
}


#
reduce_seasonal<- function(array) { 
  print("Hello World!")
  return array
}


# take one 3d arrayday reduce the 3d data to 2d
reducer2_correlation<-function(array1, array2){

  return array1+array2
}



##geom_contour
visualize_frame_level<-function(array1, array2){

  return array1+array2
}

#
visualize_frame_raw<- function(value_2d,lantitude=NULL,latidude=NULL,timestamp="",title="") { # create a function with the name my_function
  T_slice <- T_array[,,i]
  title<-xlab=date[i]
  image(longitude,rev(latitude),value_2d[,dim(value_2d)[2]:1],title,col = rev(brewer.pal(10,"RdBu")))
  
}


In [3]:
dir_path

[1] "Chlorophyll/Daily/Daily_Aqua_MODIS_chloA.2013_2022.nc"